Imports

In [1]:
import requests
import json
import pandas as pd
import os 

import plotly.express as px



### List of cities we are interested in

In [2]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]



### We look for the cities coordinates through a get request

In [5]:
responses = []

for city in cities:
    response = requests.get(f'https://nominatim.openstreetmap.org/search?q={city}&format=json')
    if response.status_code == 200:
        city_info = response.json()
        responses.append(city_info)
    else:
        print(f"Failed to get data for {city}")

print(f'Number of cities',len(responses))       

Number of cities 35


We look at the results of the first request

In [6]:
responses

[[{'place_id': 247828331,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
   'osm_type': 'way',
   'osm_id': 211285890,
   'lat': '48.6359541',
   'lon': '-1.511459954959514',
   'class': 'tourism',
   'type': 'attraction',
   'place_rank': 30,
   'importance': 0.45543655678157396,
   'addresstype': 'tourism',
   'name': 'Mont Saint-Michel',
   'display_name': 'Mont Saint-Michel, Grand Degré, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
   'boundingbox': ['48.6349172', '48.6370310', '-1.5133292', '-1.5094796']},
  {'place_id': 318698911,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
   'osm_type': 'relation',
   'osm_id': 7360493,
   'lat': '46.7798558',
   'lon': '-75.336261',
   'class': 'boundary',
   'type': 'administrative',
   'place_rank': 16,
   'importance': 0.4301414688716276,
   'addresstype': 'village',
   'name': 'Mont-Saint-Michel',
   'display_nam

### We save the first responses list in a JSON file


In [159]:
with open('city_data.json', 'w') as file:
    json.dump(responses, file, indent=4)

# Creation of a Data frame with the cities coordinates obtained from the first get request

In [8]:
city_lat = []
city_lon = []
city_order = []


responses_length = len(responses)

for i in range(0, responses_length):
    city_lat.append(responses[i][0]['lat'])
    city_lon.append(responses[i][0]['lon'])
    city_order.append(responses[i][0]['name'])

In [9]:
data = {
    'City': city_order,
    'city_lat': city_lat,
    'city_lon': city_lon
}

df = pd.DataFrame(data)

In [10]:
df.head(2)

,City,city_lat,city_lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409


Transform all numerical values to float and round(4) so that it can be used to look for data in openweather.org

In [13]:
df['city_lat'] = df['city_lat'].astype(float)
df['city_lat'] = df['city_lat'].round(4)

df['city_lon'] = df['city_lon'].astype(float)
df['city_lon'] = df['city_lon'].round(4)

df.head(2)


,City,city_lat,city_lon
0,Mont Saint-Michel,48.6360,-1.5115
1,Saint-Malo,48.6495,-2.0260


In [14]:
df_latitude = df['city_lat']
df_longitude = df['city_lon']

In [ ]:
df.to_csv("cities_coordinates.csv")

### We look for weather condition information of the 35 cities through the open weather API

In [15]:
api_key = 'aee82e4eac7ca3edf808dc34b7f2c175'

weather_responses = []

# Loop through latitude and longitude pairs
for lat, lon in zip(df_latitude, df_longitude):
    params = {'lat': lat, 
              'lon': lon,
              'appid': api_key
    }
    r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q',params=params)
    if r.status_code == 200:
        weather_info = r.json()  # Correct the variable to 'r.json()'
        weather_responses.append(weather_info)
    else:
        print(f"Failed to get data for {lat} and {lon}")


#### We save the new data in a json file

In [211]:
# Save the responses list to a JSON file
with open('weather_responses.json', 'w') as file:
    json.dump(weather_responses, file, indent=4)


In [16]:
#Test
weather_responses[0]['coord']['lon']
weather_responses[1]['main']['temp']

285.8

In [17]:
weather_length = len(weather_responses)

lat_weather = []
long_weather = []
temperature = []


responses_length = len(responses)

for i in range(0, weather_length):
    lat_weather.append(weather_responses[i]['coord']['lat'])
    long_weather.append(weather_responses[i]['coord']['lon'])
    temperature.append(weather_responses[i]['main']['temp'])

In [18]:

df['lat_weather'] = lat_weather
df['long_weather'] = long_weather

df['temperature'] = temperature
#From Kelvin to Celsius
df['temperature'] = df['temperature'].apply(lambda x: x - 273.15)

#We display df to be sure that the latitude and longitude values are the same
display(df.head(2))

,City,city_lat,city_lon,lat_weather,long_weather,temperature
0,Mont Saint-Michel,48.6360,-1.5115,48.6360,-1.5115,12.40
1,Saint-Malo,48.6495,-2.0260,48.6495,-2.0260,12.65


In [19]:
# We drop 'lat_weather','long_weather' columns because we already checked they are exactly the same values as in 'city_lat' and 'city_lon'
df = df.drop(['lat_weather','long_weather'], axis=1)


### We obtain weather forecast for each coordinate


In [20]:

api_key = 'aee82e4eac7ca3edf808dc34b7f2c175'

weather_forecast = []

# Loop through latitude and longitude pairs
for lat, lon in zip(df_latitude, df_longitude):
    params = {'lat': lat, 
              'lon': lon,
              'appid': api_key
    }
    r = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?q', params=params) 
    if r.status_code == 200:
        weather_forecast_info = r.json()  # Correct the variable to 'r.json()'
        weather_forecast.append(weather_forecast_info)
    else:
        print(f"Failed to get data for {lat} and {lon}")

We save the forecast in json file

In [219]:
weather_forecast
# Save the responses list to a JSON file
with open('weather_forecast.json', 'w') as file:
    json.dump(weather_forecast, file, indent=4)

#### We create a new dataframe with the weather forecast information

In [22]:
data = []

for i in range(0, 35):
    for x in range(0, 5):
        city_lat = weather_forecast[i]['city']['coord']['lat']
        city_lon = weather_forecast[i]['city']['coord']['lon']
        pop = weather_forecast[i]['list'][x]['pop']
        data.append({'city_lat': city_lat, 'city_lon' : city_lon, 'POP': pop})

# Create DataFrame
df_weather = pd.DataFrame(data)

# Display the DataFrame
df_weather.head(2)

,city_lat,city_lon,POP
0,48.636,-1.5115,0.0
1,48.636,-1.5115,0.2


In [ ]:
#we create a csv file with the data about POP for the next days in the 35 selected cities
df_weather.to_csv("cities_pop.csv")

Reshape of the dataframe

In [231]:
# We create a unique identifier for the probability of precipitation (POP) for the next 5 days POP 1, 2, 3, 4, 5
df_weather['pop_id'] = df_weather.groupby(['city_lat', 'city_lon']).cumcount() + 1

print(df_weather)

     city_lat  city_lon  POP  pop_id
0     48.6360   -1.5115  0.0       1
1     48.6360   -1.5115  0.0       2
2     48.6360   -1.5115  0.0       3
3     48.6360   -1.5115  0.0       4
4     48.6360   -1.5115  0.0       5
..        ...       ...  ...     ...
170   46.1591   -1.1520  0.0       1
171   46.1591   -1.1520  0.0       2
172   46.1591   -1.1520  0.0       3
173   46.1591   -1.1520  0.0       4
174   46.1591   -1.1520  0.0       5

[175 rows x 4 columns]


#### reshape the dataframe

In [232]:
#Pivot
# reshape the dataframe
df_weather_pivot = df_weather.pivot(index=['city_lat', 'city_lon'], columns='pop_id', values='POP')

# Flatten the MultiIndex columns
df_weather_pivot.columns = [f'POP_{col}' for col in df_weather_pivot.columns]

# Reset index to turn MultiIndex into columns
df_weather_pivot.reset_index(inplace=True)

print(df_weather_pivot)

    city_lat  city_lon  POP_1  POP_2  POP_3  POP_4  POP_5
0    42.5250    3.0832    0.0   0.00   0.00    0.0    0.0
1    42.9455    1.4066    0.0   0.00   0.00    0.0    0.0
2    43.1507    6.3419    0.0   0.00   0.00    0.0    0.0
3    43.2130    2.3491    0.0   0.00   0.00    0.0    0.0
4    43.2140    5.5396    0.0   0.00   0.00    0.0    0.0
5    43.2962    5.3700    0.0   0.00   0.00    0.0    0.0
6    43.4516    4.4277    0.0   0.00   0.00    0.0    0.0
7    43.4711   -1.5527    0.0   0.00   0.00    0.0    0.0
8    43.4945   -1.4737    0.0   0.00   0.00    0.0    0.0
9    43.5298    5.4475    0.0   0.00   0.00    0.0    0.0
10   43.5662    4.1915    0.0   0.00   0.00    0.0    0.0
11   43.6045    1.4442    0.0   0.00   0.00    0.0    0.0
12   43.7497    6.3286    0.0   0.00   0.00    0.0    0.0
13   43.8374    4.3601    0.0   0.00   0.00    0.0    0.0
14   43.9492    4.8059    0.0   0.00   0.00    0.0    0.0
15   44.0121    4.4197    0.0   0.00   0.00    0.0    0.0
16   44.0176  

#### Merge with the orginal dataframe with the cities' name

In [234]:
forecast = pd.merge(df, df_weather_pivot, on=['city_lat', 'city_lon'], how='left')

forecast

,City,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
0,Mont Saint-Michel,48.6360,-1.5115,19.06,0.0,0.00,0.00,0.0,0.0
1,St. Malo,49.3147,-96.9538,9.91,0.0,0.00,0.00,0.0,0.0
2,Bayeux,49.2765,-0.7025,19.14,0.0,0.00,0.00,0.0,0.0
3,Le Havre,49.4939,0.1080,17.51,0.0,0.00,0.00,0.0,0.0
4,Rouen,49.4405,1.0940,19.34,0.0,0.00,0.00,0.0,0.0
5,Paris,48.8589,2.3200,20.84,0.0,0.00,0.00,0.0,0.0
6,Amiens,49.8942,2.2957,19.20,0.2,0.37,0.20,0.0,0.0
7,Lille,50.6366,3.0635,20.22,1.0,0.57,0.23,0.0,0.0
8,Strasbourg,48.5846,7.7507,22.96,0.2,0.00,0.00,0.0,0.0
9,Château du Haut-Kœnigsbourg,48.2494,7.3443,19.15,0.2,0.20,0.00,0.0,0.0


Save the data to csv file

In [241]:
city_id = [i for i in range(1,36)]
forecast.insert(loc=0, column='city_id', value=city_id)
forecast

,city_id,City,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
0,1,Mont Saint-Michel,48.6360,-1.5115,19.06,0.0,0.00,0.00,0.0,0.0
1,2,St. Malo,49.3147,-96.9538,9.91,0.0,0.00,0.00,0.0,0.0
2,3,Bayeux,49.2765,-0.7025,19.14,0.0,0.00,0.00,0.0,0.0
3,4,Le Havre,49.4939,0.1080,17.51,0.0,0.00,0.00,0.0,0.0
4,5,Rouen,49.4405,1.0940,19.34,0.0,0.00,0.00,0.0,0.0
5,6,Paris,48.8589,2.3200,20.84,0.0,0.00,0.00,0.0,0.0
6,7,Amiens,49.8942,2.2957,19.20,0.2,0.37,0.20,0.0,0.0
7,8,Lille,50.6366,3.0635,20.22,1.0,0.57,0.23,0.0,0.0
8,9,Strasbourg,48.5846,7.7507,22.96,0.2,0.00,0.00,0.0,0.0
9,10,Château du Haut-Kœnigsbourg,48.2494,7.3443,19.15,0.2,0.20,0.00,0.0,0.0


We save the data in csv file

In [242]:
forecast.to_csv("cities_forecast.csv")

In [23]:
forecast_df = pd.read_csv('/Users/agustindei/Documents/Jedha/Full_Stack/Projets/Kayak/cities_forecast.csv')

In [25]:
fig = px.scatter_mapbox(forecast_df, lat="city_lat", lon="city_lon", color="City",
                        zoom=4, mapbox_style="carto-positron")# color_discrete_sequence=True)
fig.show()

In [26]:
forecast_df.head(2)

,Unnamed: 0,city_id,City,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
0,0,1,Mont Saint-Michel,48.6360,-1.5115,19.06,0.0,0.0,0.0,0.0,0.0
1,1,2,St. Malo,49.3147,-96.9538,9.91,0.0,0.0,0.0,0.0,0.0


### TOP 5 Cities by weather and top 20 hotels within those cities

In [28]:
top_cities = forecast_df.sort_values(by='temperature', ascending=False).head(5)
top_cities

,Unnamed: 0,city_id,City,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
25,25,26,Aigues-Mortes,43.5662,4.1915,27.95,0.0,0.0,0.0,0.0,0.0
26,26,27,Saintes-Maries-de-la-Mer,43.4516,4.4277,27.59,0.0,0.0,0.0,0.0,0.0
27,27,28,Collioure,42.5250,3.0832,27.46,0.0,0.0,0.0,0.0,0.0
24,24,25,Nîmes,43.8374,4.3601,27.22,0.0,0.0,0.0,0.0,0.0
20,20,21,Marseille,43.2962,5.3700,26.65,0.0,0.0,0.0,0.0,0.0


In [30]:
fig = px.scatter_mapbox(forecast_df, lat="city_lat", lon="city_lon", color="POP_1",
                        zoom=4, mapbox_style="carto-positron",title='Precipitation probability from 0 to 1 for all the cities', color_discrete_sequence=True)

fig.update_layout(width=550, height=600)
fig.show()

fig = px.scatter_mapbox(top_cities, lat="city_lat", lon="city_lon", color="POP_1",
                        zoom=4, mapbox_style="carto-positron",title='Precipitation probability from 0 to 1 of top 5 warmest cities in France', color_discrete_sequence=True)

fig.update_layout(width=700, height=500)
fig.show()

Only cities in the nord and east of France are expected to be under raining weather conditions.

The five top warmest cities at the moment are expected also to have no precipitation. They are the top 5 destionations: Aigues-Mortes, Saintes-Maries-de-la-Mer, Collioure, Nîmes and Marseille

### We use the data obtained from boooking website that we have saved in json file to create a new Dataframe about hotels in these cities

In [31]:
path_json_hotels = '/Users/agustindei/Documents/Jedha/Full_Stack/Projets/Kayak/booking_scraping/booking5.json'

df_hotels = pd.read_json(path_json_hotels)
df_hotels.head()


,city,hotel,url,note,coordinates,description
0,Paris,Hôtel Diva Opera,https://www.booking.com/hotel/fr/gotty-opera.f...,"8,0","48.87327211,2.34507881",L'Hotel Diva Opéra est situé dans le 9ème arro...
1,Paris,Tipi,https://www.booking.com/hotel/fr/tipi.fr.html?...,"7,1","48.83548585,2.32537396","Situé dans le centre de Paris, cet hôtel est i..."
2,Paris,Hôtel Du Mont Dore Batignolles,https://www.booking.com/hotel/fr/hotel-du-mont...,"8,0","48.88370912,2.32275814",L'Hôtel Du Mont Dore Batignolles se trouve à 1...
3,Paris,Hôtel de Genève,https://www.booking.com/hotel/fr/de-geneve.fr....,"7,5","48.87774274,2.32753918",Situé à 300 mètres de la gare Saint-Lazare et ...
4,Paris,B&B HOTEL Paris 17 Batignolles,https://www.booking.com/hotel/fr/parisboulevar...,"8,0","48.89431407,2.31234401",Le B&B Hôtel Paris 17 Batignolles est situé da...


We create the latitude and longitude columns by splitting the coordinates column

In [32]:
df_hotels[['hotel_latitude', 'hotel_longitude']] = df_hotels['coordinates'].str.split(',', expand=True)

# Convert the latitude and longitude to float
df_hotels['hotel_latitude'] = df_hotels['hotel_latitude'].astype(float)
df_hotels['hotel_longitude'] = df_hotels['hotel_longitude'].astype(float)
df_hotels.drop(columns='coordinates', axis=1) 
df_hotels.head()

,city,hotel,url,note,coordinates,description,hotel_latitude,hotel_longitude
0,Paris,Hôtel Diva Opera,https://www.booking.com/hotel/fr/gotty-opera.f...,"8,0","48.87327211,2.34507881",L'Hotel Diva Opéra est situé dans le 9ème arro...,48.873272,2.345079
1,Paris,Tipi,https://www.booking.com/hotel/fr/tipi.fr.html?...,"7,1","48.83548585,2.32537396","Situé dans le centre de Paris, cet hôtel est i...",48.835486,2.325374
2,Paris,Hôtel Du Mont Dore Batignolles,https://www.booking.com/hotel/fr/hotel-du-mont...,"8,0","48.88370912,2.32275814",L'Hôtel Du Mont Dore Batignolles se trouve à 1...,48.883709,2.322758
3,Paris,Hôtel de Genève,https://www.booking.com/hotel/fr/de-geneve.fr....,"7,5","48.87774274,2.32753918",Situé à 300 mètres de la gare Saint-Lazare et ...,48.877743,2.327539
4,Paris,B&B HOTEL Paris 17 Batignolles,https://www.booking.com/hotel/fr/parisboulevar...,"8,0","48.89431407,2.31234401",Le B&B Hôtel Paris 17 Batignolles est situé da...,48.894314,2.312344


# Final DataFrame : merging of the forecaste dataframe and the hotels dataframe

### Before merging the forecast dataframe and the hotels dataframe we check if there are any difference between the cities names 

In [123]:
# Get unique cities from both DataFrames
cities_in_forecast_df = set(forecast_df['city'].unique())
cities_in_df_hotels = set(df_hotels['city'].unique())

# Find differences
cities_only_in_forecast_df = cities_in_forecast_df - cities_in_df_hotels
cities_only_in_df_hotels = cities_in_df_hotels - cities_in_forecast_df

# Display the differences
print("Cities only in forecast_df:", cities_only_in_forecast_df)
print("Cities only in df_hotels:", cities_only_in_df_hotels)

Cities only in forecast_df: {'Ariège', 'St. Malo', 'Uzès', 'Mont Saint-Michel', 'Besançon', 'Aigues-Mortes', 'Bormes-les-Mimosas', 'Château du Haut-Kœnigsbourg', 'Aix-en-Provence'}
Cities only in df_hotels: {'Ariege', 'Bormes les Mimosas', 'Aigues Mortes', 'Aix en Provence', 'Besancon', 'Uzes', 'Chateau du Haut Koenigsbourg', 'Mont Saint Michel', 'St Malo'}


### We correct the differences before merging

In [124]:
#Changes in forecast_df

forecast_df = forecast_df.loc[:, ~forecast_df.columns.str.contains('^Unnamed')]
forecast_df = forecast_df.rename(columns={'City': 'city'})
forecast_df['city'] = forecast_df['city'].replace({'Saintes-Maries-de-la-Mer': 'Les Saintes-Maries-de-la-Mer'})

#Changes in df_hotels

# Create a dictionary to map city names from df_hotels to forecast_df
city_replacements = {
    'Ariege': 'Ariège',
    'Bormes les Mimosas': 'Bormes-les-Mimosas',
    'Aigues Mortes': 'Aigues-Mortes',
    'Aix en Provence': 'Aix-en-Provence',
    'Besancon': 'Besançon',
    'Uzes': 'Uzès',
    'Chateau du Haut Koenigsbourg': 'Château du Haut-Kœnigsbourg',
    'Mont Saint Michel': 'Mont Saint-Michel',
    'St Malo': 'St. Malo'
}

# Replace city names in df_hotels
df_hotels['city'] = df_hotels['city'].replace(city_replacements)


# We obtain our final dataframe

In [125]:
df_final = pd.merge(df_hotels, forecast_df, on='city', how='left')

df_final.head()

,city,hotel,url,note,coordinates,description,hotel_latitude,hotel_longitude,city_id,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
0,Paris,Hôtel Diva Opera,https://www.booking.com/hotel/fr/gotty-opera.f...,"8,0","48.87327211,2.34507881",L'Hotel Diva Opéra est situé dans le 9ème arro...,48.873272,2.345079,6,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
1,Paris,Tipi,https://www.booking.com/hotel/fr/tipi.fr.html?...,"7,1","48.83548585,2.32537396","Situé dans le centre de Paris, cet hôtel est i...",48.835486,2.325374,6,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
2,Paris,Hôtel Du Mont Dore Batignolles,https://www.booking.com/hotel/fr/hotel-du-mont...,"8,0","48.88370912,2.32275814",L'Hôtel Du Mont Dore Batignolles se trouve à 1...,48.883709,2.322758,6,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
3,Paris,Hôtel de Genève,https://www.booking.com/hotel/fr/de-geneve.fr....,"7,5","48.87774274,2.32753918",Situé à 300 mètres de la gare Saint-Lazare et ...,48.877743,2.327539,6,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
4,Paris,B&B HOTEL Paris 17 Batignolles,https://www.booking.com/hotel/fr/parisboulevar...,"8,0","48.89431407,2.31234401",Le B&B Hôtel Paris 17 Batignolles est situé da...,48.894314,2.312344,6,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0


we check the shape

In [126]:
forecast_df.shape

(35, 10)

Final adjustments

In [87]:
# Move 'city_id' to the first position
cols = ['city_id'] + [col for col in df_final.columns if col != 'city_id']

# Reassign the dataframe with the new column order
df_final = df_final[cols]
df_final.head()

,city_id,city,hotel,url,note,coordinates,description,hotel_latitude,hotel_longitude,city_lat,city_lon,temperature,POP_1,POP_2,POP_3,POP_4,POP_5
0,6.0,Paris,Hôtel Diva Opera,https://www.booking.com/hotel/fr/gotty-opera.f...,"8,0","48.87327211,2.34507881",L'Hotel Diva Opéra est situé dans le 9ème arro...,48.873272,2.345079,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
1,6.0,Paris,Tipi,https://www.booking.com/hotel/fr/tipi.fr.html?...,"7,1","48.83548585,2.32537396","Situé dans le centre de Paris, cet hôtel est i...",48.835486,2.325374,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
2,6.0,Paris,Hôtel Du Mont Dore Batignolles,https://www.booking.com/hotel/fr/hotel-du-mont...,"8,0","48.88370912,2.32275814",L'Hôtel Du Mont Dore Batignolles se trouve à 1...,48.883709,2.322758,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
3,6.0,Paris,Hôtel de Genève,https://www.booking.com/hotel/fr/de-geneve.fr....,"7,5","48.87774274,2.32753918",Situé à 300 mètres de la gare Saint-Lazare et ...,48.877743,2.327539,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0
4,6.0,Paris,B&B HOTEL Paris 17 Batignolles,https://www.booking.com/hotel/fr/parisboulevar...,"8,0","48.89431407,2.31234401",Le B&B Hôtel Paris 17 Batignolles est situé da...,48.894314,2.312344,48.8589,2.32,20.84,0.0,0.0,0.0,0.0,0.0


### We save all the data in the final df in a csv file

In [140]:
df_final.to_csv("df_final_kayak_project.csv")